In [ ]:
!pip install pennylane pennylane-lightning

In [ ]:
from qepc.sports.nba.player_data import load_raw_player_data; df = load_raw_player_data(); print(df.head())

In [ ]:
from qepc.sports.nba.props_adjust import adjust_prop; print(adjust_prop(20, 105, 35))

In [ ]:
from qepc.core.simulator import run_qepc_simulation;

In [6]:
from nba_api.stats.static import teams
print(teams.get_teams()[0])  # Shows a team like Lakers

{'id': 1610612737, 'full_name': 'Atlanta Hawks', 'abbreviation': 'ATL', 'nickname': 'Hawks', 'city': 'Atlanta', 'state': 'Georgia', 'year_founded': 1949}


In [7]:
!pip install nba_api requests

In [8]:
# ... (keep your existing code at top)

import requests  # For Balldontlie

BALLDONTLIE_API_KEY = "c5ae7df3-682e-450c-b47e-f7e91396379e"  # Your key

def load_live_injuries(local_file="data/Injury_Overrides.csv"):
    """
    Fetches live injuries from API.
    - Tries nba_api Rotowire first (live updates).
    - Backup: Balldontlie API.
    - Fallback: Local CSV.
    """
    try:
        # nba_api Rotowire (live)
        from nba_api.live.nba.endpoints import scoreboard
        board = scoreboard.ScoreBoard()
        games = board.games.get_dict()
        # Extract injuries (simplified; Rotowire has full)
        injuries = []  # Loop games for player status
        for game in games:
            for team in [game['homeTeam'], game['awayTeam']]:
                for player in team['players']:
                    if player.get('status') != 'ACTIVE':
                        injuries.append({'PlayerName': player['name'], 'Team': team['teamTricode'], 'Status': player.get('status', 'Unknown')})
        df = pd.DataFrame(injuries)
        if not df.empty:
            print(f"Fetched {len(df)} live injuries from Rotowire/nba_api.")
            return df
    except Exception as e:
        print(f"nba_api error: {e}. Trying Balldontlie.")
    
    try:
        # Balldontlie backup (live with key)
        url = "https://api.balldontlie.io/v1/injuries"
        headers = {"Authorization": BALLDONTLIE_API_KEY}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()['data']
            df = pd.DataFrame(data)
            df = df[['player', 'team', 'status']]  # Adjust cols
            df.rename(columns={'player': 'PlayerName', 'team': 'Team', 'status': 'Status'}, inplace=True)
            print(f"Fetched {len(df)} live injuries from Balldontlie.")
            return df
    except Exception as e:
        print(f"Balldontlie error: {e}. Using local file.")
    
    # Fallback local
    df = pd.read_csv(local_file)
    print(f"Used local {len(df)} injuries.")
    return df

In [9]:
!pip install requests  # For Balldontlie

ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [ ]:
!pip install requests "